## Import tools

In [3]:
import pandas as pd
import requests
import re
import os
from bs4 import BeautifulSoup as bs
import os.path as path
from PIL import Image

## Fetching file

In [199]:
path = "C:/Users/Chris/Desktop/DTU/4. Semester/02466 - Project Work/RedditDataWithLinks_Master.csv"
posts_df = pd.read_csv(path)


C:\Users\Chris\AppData\Local\Temp\ipykernel_23984\2805157975.py:4: DtypeWarning: Columns (19,21,22,25,26,27,28,29,30,32,33,35,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  posts_df = pd.read_csv(path)


##### Preview samples

In [187]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 25)
print(posts_df)


        Subreddit SubredditID  \
0       EarthPorn    t5_2sbq3   
1       EarthPorn    t5_2sbq3   
2       EarthPorn    t5_2sbq3   
3       EarthPorn    t5_2sbq3   
4       EarthPorn    t5_2sbq3   
...           ...         ...   
763350  EarthPorn    t5_2sbq3   
763351  EarthPorn    t5_2sbq3   
763352  EarthPorn    t5_2sbq3   
763353  EarthPorn    t5_2sbq3   
763354  EarthPorn    t5_2sbq3   

                                                                                                                                                                                            PostTitle  \
0                                                                                                                                                                   Kalsoy, Faroe Islands [2214x1548]   
1                                                                                                                          The shimmering slopes of the pristine Austrian Alps [1024x1023] © 2010 Ben   


#### Clean up data frame

##### Get list of all column names

In [188]:
colunmNames = posts_df.columns.to_list()
print(colunmNames)

['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'TimeString', 'PostTime', 'Username', 'ViewCount', 'Upvotes', 'Ups', 'Downs', 'UpvoteRatio', 'CommentNumber', 'Edited', 'ImageDomain', 'ImageURL', 'Permalink', 'IsTextPost', 'PostText', 'SelfTextHTML', 'UserCreatedTime', 'AuthorName', 'Distinguished', 'LinkFlairClass', 'LinkFlairText', 'AuthorBirthdate', 'IsVideo', 'IsMeta', 'IsOriginal', 'IsRedditMediaDomain', 'IsCrosspostable', 'CrosspostsNumber', 'CrosspostParent', 'CrosspostParentList', 'SubRedditSubscribers', 'OnlyMedia', 'EmbeddedMedia', 'SecureEmbeddedMedia', 'Media', 'SecureMedia', 'Thumbnail', 'Stickied', 'Archived', 'IsHideScore', 'Pinned', 'PostHint', 'Preview', 'CommentsLocked', 'NSFW', 'NewURL']


##### Remove unwanted columns

In [189]:
# posts_df.drop(columns = ['author_flair_css_class', 'author_flair_text'], inplace = True)

posts_df.drop(columns = ['quarantine', 'link_flair_template_id', 'edited_on', 'call_to_action', 'subreddit_name_prefixed', 'discussion_type', 'post_categories', 'content_categories', 'category', 'can_gild', 'author_flair_template_id', 'from', 'from_kind', 'from_id', 'hidden', 'likes', 'retrieved_utc', 'created', 'report_reasons', 'approved_by', 'saved', 'removal_reason', 'num_reports', 'author_flair_css_class', 'author_flair_text', 'gilded', 'mod_reports', 'user_reports', 'brand_safe', 'contest_mode', 'spoiler', 'suggested_sort', 'author_flair_richtext', 'author_flair_type', 'link_flair_richtext', 'link_flair_text_color', 'link_flair_type', 'rte_mode', 'subreddit_type', 'thumbnail_height', 'thumbnail_width', 'author_flair_background_color', 'author_flair_text_color', 'author_patreon_flair', 'gildings', 'is_robot_indexable', 'link_flair_background_color', 'send_replies', 'no_follow', 'updated_utc', 'all_awardings', 'allow_live_comments', 'author_premium', 'awarders', 'total_awards_received', 'treatment_tags', 'is_created_from_ads_ui', 'parent_whitelist_status', 'pwls', 'url_overridden_by_dest', 'whitelist_status', 'wls', 'removed_by_category', 'approved_at_utc', 'banned_at_utc', 'removed_by', 'top_awarded_type', 'retrieved_on'], inplace = True, errors='ignore')

colunmNames = posts_df.columns.to_list()
print(colunmNames)
print(len(colunmNames))
del colunmNames

['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'TimeString', 'PostTime', 'Username', 'ViewCount', 'Upvotes', 'Ups', 'Downs', 'UpvoteRatio', 'CommentNumber', 'Edited', 'ImageDomain', 'ImageURL', 'Permalink', 'IsTextPost', 'PostText', 'SelfTextHTML', 'UserCreatedTime', 'AuthorName', 'Distinguished', 'LinkFlairClass', 'LinkFlairText', 'AuthorBirthdate', 'IsVideo', 'IsMeta', 'IsOriginal', 'IsRedditMediaDomain', 'IsCrosspostable', 'CrosspostsNumber', 'CrosspostParent', 'CrosspostParentList', 'SubRedditSubscribers', 'OnlyMedia', 'EmbeddedMedia', 'SecureEmbeddedMedia', 'Media', 'SecureMedia', 'Thumbnail', 'Stickied', 'Archived', 'IsHideScore', 'Pinned', 'PostHint', 'Preview', 'CommentsLocked', 'NSFW', 'NewURL']
50


##### Change column names and reorder columns

1. Create dictionary - 'old name' : 'new name'

In [190]:
column_names = {'id' : 'PostID',
                'subreddit' : 'Subreddit',
                'subreddit_id' : 'SubredditID',
                'created_utc' : 'PostTime',
                'title' : 'PostTitle',
                'author' : 'Username',
                'author_created_utc' : 'UserCreatedTime',
                'author_fullname' : 'AuthorName', 
                'domain' : 'ImageDomain',
                'full_link' : 'Link',
                'is_self' : 'IsTextPost',
                'media_embed' : 'EmbeddedMedia',
                'secure_media_embed' : 'SecureEmbeddedMedia',
                'num_comments' : 'CommentNumber', 
                'over_18' : 'NSFW',
                'permalink' : 'Permalink', 
                'score' : 'Upvotes', 
                'selftext' : 'PostText', 
                'thumbnail' : 'Thumbnail',
                'url' : 'ImageURL',
                'media' : 'Media',
                'secure_media' : 'SecureMedia',
                'stickied' : 'Stickied',
                'locked' : 'CommentsLocked',
                'post_hint' : 'PostHint',
                'preview' : 'Preview',
                'is_crosspostable' : 'IsCrosspostable',
                'is_reddit_media_domain' : 'IsRedditMediaDomain',
                'is_video' : 'IsVideo',
                'num_crossposts' : 'CrosspostsNumber', 
                'pinned' : 'Pinned',
                'crosspost_parent' : 'CrosspostParent',
                'crosspost_parent_list' : 'CrosspostParentList',
                'is_meta' : 'IsMeta',
                'is_original_content' : 'IsOriginal',
                'media_only' : 'OnlyMedia', 
                'subreddit_subscribers' : 'SubRedditSubscribers',
                'media_metadata' : 'MediaMetadata', 
                'upvote_ratio' : 'UpvoteRatio', 
                'gallery_data' : 'GalleryData', 
                'is_gallery' : 'IsGallery', 
                'author_cakeday' : 'AuthorBirthdate',
                'edited' : 'Edited', 
                'view_count' : 'ViewCount', 
                'author_id' : 'AuthorID',
                'og_description' : 'OGDescription',
                'og_title' : 'OGTitle',
                'utc_datetime_str' : 'TimeString',
                'ups' : 'Ups',
                'downs' : 'Downs',
                'selftext_html' : 'SelfTextHTML',
                'distinguished' : 'Distinguished',
                'link_flair_css_class' : 'LinkFlairClass',
                'link_flair_text' : 'LinkFlairText',
                'archived' : 'Archived',
                'hide_score' : 'IsHideScore'}

2. Rename columns using dictionary

In [191]:
posts_tidy_df = posts_df.rename(columns = column_names)
# Check to see if columns have been renamed
posts_tidy_df.columns 

Index(['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'TimeString',
       'PostTime', 'Username', 'ViewCount', 'Upvotes', 'Ups', 'Downs',
       'UpvoteRatio', 'CommentNumber', 'Edited', 'ImageDomain', 'ImageURL',
       'Permalink', 'IsTextPost', 'PostText', 'SelfTextHTML',
       'UserCreatedTime', 'AuthorName', 'Distinguished', 'LinkFlairClass',
       'LinkFlairText', 'AuthorBirthdate', 'IsVideo', 'IsMeta', 'IsOriginal',
       'IsRedditMediaDomain', 'IsCrosspostable', 'CrosspostsNumber',
       'CrosspostParent', 'CrosspostParentList', 'SubRedditSubscribers',
       'OnlyMedia', 'EmbeddedMedia', 'SecureEmbeddedMedia', 'Media',
       'SecureMedia', 'Thumbnail', 'Stickied', 'Archived', 'IsHideScore',
       'Pinned', 'PostHint', 'Preview', 'CommentsLocked', 'NSFW', 'NewURL'],
      dtype='object')

3. Reorder columns

In [192]:
#posts_tidy_df = posts_tidy_df[['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'PostTime', 'Username', 'Upvotes', 'CommentNumber', 'ImageDomain', 'ImageURL', 'UserCreatedTime', 'AuthorName', 'Permalink', 'Link', 'IsTextPost', 'PostText', 'EmbeddedMedia', 'Thumbnail', 'NSFW']]
posts_tidy_df = posts_tidy_df[['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'TimeString', 'PostTime', 'Username', 'ViewCount', 'Upvotes', 'Ups', 'Downs', 'UpvoteRatio', 'CommentNumber', 'Edited', 'ImageDomain', 'ImageURL', 'Permalink', 'IsTextPost', 'PostText', 'SelfTextHTML', 'UserCreatedTime', 'AuthorName', 'Distinguished', 'LinkFlairClass', 'LinkFlairText', 'AuthorBirthdate', 'IsVideo', 'IsMeta', 'IsOriginal', 'IsRedditMediaDomain', 'IsCrosspostable', 'CrosspostsNumber', 'CrosspostParent', 'CrosspostParentList', 'SubRedditSubscribers', 'OnlyMedia', 'EmbeddedMedia', 'SecureEmbeddedMedia', 'Media', 'SecureMedia', 'Thumbnail', 'Stickied', 'Archived', 'IsHideScore', 'Pinned', 'PostHint', 'Preview', 'CommentsLocked', 'NSFW', 'NewURL']]
                                       

#posts_reordered_df = posts_renamed_df[['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'PostTime', 'Username', 'ViewCount', 'Upvotes', 'UpvoteRatio', 'CommentNumber', 'Edited', 'OGDescription', 'OGTitle', 'ImageDomain', 'ImageURL', 'Permalink', 'Link', 'IsTextPost', 'PostText', 'UserCreatedTime', 'AuthorID', 'AuthorName', 'AuthorBirthdate', 'IsVideo', 'IsMeta', 'IsOriginal', 'IsGallery', 'GalleryData', 'IsRedditMediaDomain', 'IsCrosspostable', 'CrosspostsNumber', 'CrosspostParent', 'CrosspostParentList', 'SubRedditSubscribers', 'OnlyMedia', 'MediaMetadata', 'EmbeddedMedia', 'SecureEmbeddedMedia', 'Media', 'SecureMedia', 'Thumbnail', 'Stickied', 'Pinned', 'PostHint', 'Preview', 'CommentsLocked', 'NSFW']]

In [193]:
print(posts_tidy_df)

        Subreddit SubredditID  \
0       EarthPorn    t5_2sbq3   
1       EarthPorn    t5_2sbq3   
2       EarthPorn    t5_2sbq3   
3       EarthPorn    t5_2sbq3   
4       EarthPorn    t5_2sbq3   
...           ...         ...   
763350  EarthPorn    t5_2sbq3   
763351  EarthPorn    t5_2sbq3   
763352  EarthPorn    t5_2sbq3   
763353  EarthPorn    t5_2sbq3   
763354  EarthPorn    t5_2sbq3   

                                                                                                                                                                                            PostTitle  \
0                                                                                                                                                                   Kalsoy, Faroe Islands [2214x1548]   
1                                                                                                                          The shimmering slopes of the pristine Austrian Alps [1024x1023] © 2010 Ben   


#### Fix image URLS

In [194]:
#create column for fixed urls
posts_tidy_df = posts_tidy_df.reindex(columns = posts_tidy_df.columns.tolist() + ['NewURL'])

posts_tidy_df = posts_tidy_df.head(50)

In [196]:
for index, row in posts_tidy_df.iterrows():
    if row['ImageDomain'] == 'flickr.com':
        print(row['ImageURL'])
        r = requests.get(row['ImageURL'])
        soup = bs(r.content)
        images = re.findall(r'(\/\/live\.staticflickr\.com\/[0-9][0-9][0-9][0-9]\/[a-zA-Z0-9_]+\.(?:png|jpg|jpeg|gif|png|svg))', str(soup))
        
        for image in images:
            image_url = image
            break
        print(image_url)
        posts_tidy_df.at[index, 'NewURL'] = image_url
    elif row['ImageDomain'] == 'imgur.com':
        posts_tidy_df.at[index, 'NewURL'] = re.sub(r'http://imgur.com', 'http://i.imgur.com', row['ImageURL']) + '.jpg'
    elif row['ImageDomain'] == 'i.imgur.com':
        posts_tidy_df.at[index, 'NewURL'] = row['ImageURL']
    else:
        continue

# Save the new DataFrame as a CSV file
posts_tidy_df.to_csv("C:/Users/Chris/Desktop/DTU/4. Semester/02466 - Project Work/RedditDataNewURL.csv", index=False)

http://www.flickr.com/photos/visbeek/6772214933/lightbox/
//live.staticflickr.com/7146/6772214933_a5d596dc4a_b.jpg
http://www.flickr.com/photos/devon_buckingham/7121215233/in/set-72157629196222538/lightbox/
//live.staticflickr.com/2823/33143780524_a94028c764.jpg
http://www.flickr.com/photos/desertsky/21597748/lightbox/
//live.staticflickr.com/2823/33143780524_a94028c764.jpg
http://www.flickr.com/photos/worldofarun/5690199815/sizes/l/in/set-72157622764231740/
//live.staticflickr.com/5229/5690199815_87c9e2ac3f_m.jpg
http://www.flickr.com/photos/46418126@N02/7120325487/in/photostream/lightbox/
//live.staticflickr.com/5229/5690199815_87c9e2ac3f_m.jpg
http://www.flickr.com/photos/gmj49/6971856078/
//live.staticflickr.com/8164/6971856078_1fb002b26d_b.jpg
http://www.flickr.com/photos/parfeniuk/5766183609/in/set-72157626675557795/lightbox/
//live.staticflickr.com/2224/5766183609_969d89454e_b.jpg
http://www.flickr.com/photos/parfeniuk/5766183609/in/photostream/lightbox/
//live.staticflickr.com/

## Save data frame and images // START FROM HERE IF YOU HAVE MASTER FILE

#### View data frame

In [4]:
path = "C:/Users/Chris/Desktop/DTU/4. Semester/02466 - Project Work/RedditDataWithLinks_Master.csv"
posts_tidy_df = pd.read_csv(path)

pd.set_option('display.max_columns', None)
posts_tidy_df


C:\Users\Chris\AppData\Local\Temp\ipykernel_37664\2068525192.py:2: DtypeWarning: Columns (19,21,22,25,26,27,28,29,30,32,33,35,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  posts_tidy_df = pd.read_csv(path)


,Subreddit,SubredditID,PostTitle,PostID,TimeString,PostTime,Username,ViewCount,Upvotes,Ups,...,Thumbnail,Stickied,Archived,IsHideScore,Pinned,PostHint,Preview,CommentsLocked,NSFW,NewURL
0,EarthPorn,t5_2sbq3,"Kalsoy, Faroe Islands [2214x1548]",swun9,2012-04-28 15:32:12,1335627132,jonpacker,NaN,115,124.0,...,http://e.thumbs.redditmedia.com/oCCuPAV58aNOHEuL.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://i.imgur.com/x0PIq.jpg
1,EarthPorn,t5_2sbq3,The shimmering slopes of the pristine Austrian Alps [1024x1023] © 2010 Ben,swtuj,2012-04-28 15:08:04,1335625684,kjoneslol,NaN,9,13.0,...,http://e.thumbs.redditmedia.com/pRiuG_KieSX9rFCL.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,//live.staticflickr.com/7146/6772214933_a5d596dc4a_b.jpg
2,EarthPorn,t5_2sbq3,"Moon and Clouds in Eleuthera, Bahamas [OC] [900x700]",swteg,2012-04-28 14:54:45,1335624885,[deleted],NaN,3,3.0,...,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,//live.staticflickr.com/2823/33143780524_a94028c764.jpg
3,EarthPorn,t5_2sbq3,"Brennisteinsalda volcano, Iceland [1024x768]",swr1z,2012-04-28 13:35:01,1335620101,liverman,NaN,3,3.0,...,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://i.imgur.com/8hP8h.jpg
4,EarthPorn,t5_2sbq3,"View from Val Thorens, France [5604x2082]",swqhw,2012-04-28 13:11:35,1335618695,vhs86,NaN,12,14.0,...,http://d.thumbs.redditmedia.com/ChEKCp4SEM8aZLCE.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://i.imgur.com/KmyH8.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763350,EarthPorn,t5_2sbq3,What it's like to sit at the edge of Half Dome looking down into Yosemite Valley [OC] [3264x2448],6v2l1x,2017-08-21 12:30:05,1503318605,[deleted],NaN,6,NaN,...,default,False,False,False,NaN,NaN,NaN,False,False,NaN
763351,EarthPorn,t5_2sbq3,"Shot on Galaxy S7. Tučepi, Croatia",6v2ksx,2017-08-21 12:28:31,1503318511,[deleted],NaN,1,NaN,...,default,False,False,False,NaN,NaN,NaN,False,False,NaN
763352,EarthPorn,t5_2sbq3,"I hitchhiked around Iceland 🇮🇸 for four weeks, stayed up really late to take this pic of Kirkjufell with my Canon 5D4. This was featured in the latest ep of Game of Thrones [OC] [1080x1350]",6v2kdx,2017-08-21 12:26:11,1503318371,[deleted],NaN,1,NaN,...,default,False,False,False,NaN,NaN,NaN,False,False,http://i.imgur.com/YTOAnec.jpg
763353,EarthPorn,t5_2sbq3,"Morning at Havel, Milower Land, Brandenburg (OC)[1920x1080]",6v2hbi,2017-08-21 12:09:10,1503317350,MadameLumiere,NaN,4,NaN,...,default,False,False,False,NaN,NaN,NaN,False,False,NaN


#### Cleaning data

In [7]:
print(len(posts_tidy_df))

posts_tidy_df.drop(posts_tidy_df[posts_tidy_df['Ups'] + posts_tidy_df['Downs']  <= 10].index, inplace = True)

posts_tidy_df['UpvoteRatio'] = (posts_tidy_df['Upvotes'] / posts_tidy_df['Ups'])

print(len(posts_tidy_df))
print(posts_tidy_df['NewURL'].isna().sum())


pd.set_option('display.max_columns', None)
posts_tidy_df

575058
575058
461718


,Subreddit,SubredditID,PostTitle,PostID,TimeString,PostTime,Username,ViewCount,Upvotes,Ups,Downs,UpvoteRatio,CommentNumber,Edited,ImageDomain,ImageURL,Permalink,IsTextPost,PostText,SelfTextHTML,UserCreatedTime,AuthorName,Distinguished,LinkFlairClass,LinkFlairText,AuthorBirthdate,IsVideo,IsMeta,IsOriginal,IsRedditMediaDomain,IsCrosspostable,CrosspostsNumber,CrosspostParent,CrosspostParentList,SubRedditSubscribers,OnlyMedia,EmbeddedMedia,SecureEmbeddedMedia,Media,SecureMedia,Thumbnail,Stickied,Archived,IsHideScore,Pinned,PostHint,Preview,CommentsLocked,NSFW,NewURL
0,EarthPorn,t5_2sbq3,"Kalsoy, Faroe Islands [2214x1548]",swun9,2012-04-28 15:32:12,1335627132,jonpacker,NaN,115,124.0,9.0,0.927419,2,False,i.imgur.com,http://i.imgur.com/x0PIq.jpg,/r/EarthPorn/comments/swun9/kalsoy_faroe_islands_2214x1548/,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,http://e.thumbs.redditmedia.com/oCCuPAV58aNOHEuL.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://i.imgur.com/x0PIq.jpg
1,EarthPorn,t5_2sbq3,The shimmering slopes of the pristine Austrian Alps [1024x1023] © 2010 Ben,swtuj,2012-04-28 15:08:04,1335625684,kjoneslol,NaN,9,13.0,4.0,0.692308,0,False,flickr.com,http://www.flickr.com/photos/visbeek/6772214933/lightbox/,/r/EarthPorn/comments/swtuj/the_shimmering_slopes_of_the_pristine_austrian/,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,http://e.thumbs.redditmedia.com/pRiuG_KieSX9rFCL.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,//live.staticflickr.com/7146/6772214933_a5d596dc4a_b.jpg
4,EarthPorn,t5_2sbq3,"View from Val Thorens, France [5604x2082]",swqhw,2012-04-28 13:11:35,1335618695,vhs86,NaN,12,14.0,2.0,0.857143,3,False,imgur.com,http://imgur.com/KmyH8,/r/EarthPorn/comments/swqhw/view_from_val_thorens_france_5604x2082/,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,http://d.thumbs.redditmedia.com/ChEKCp4SEM8aZLCE.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://i.imgur.com/KmyH8.jpg
6,EarthPorn,t5_2sbq3,"Devetashka Cave, Bulgaria [1600x1071]",swq3t,2012-04-28 12:54:45,1335617685,Mind_Virus,NaN,1155,1950.0,795.0,0.592308,35,False,i.imgur.com,http://i.imgur.com/wPGUn.jpg,/r/EarthPorn/comments/swq3t/devetashka_cave_bulgaria_1600x1071/,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,http://d.thumbs.redditmedia.com/sZEkuz4BrWejGKZQ.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://i.imgur.com/wPGUn.jpg
9,EarthPorn,t5_2sbq3,Yellowstone National Park [1062x752],swphj,2012-04-28 12:24:30,1335615870,Shooters,NaN,19,21.0,2.0,0.904762,1,False,th05.deviantart.net,http://th05.deviantart.net/fs70/PRE/i/2012/118/f/f/yellowstone_national_park__5_by_melonamen-d4xvrmj.jpg,/r/EarthPorn/comments/swphj/yellowstone_national_park_1062x752/,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,http://f.thumbs.redditmedia.com/bu0UAYiaCtA7DjR5.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763350,EarthPorn,t5_2sbq3,What it's like to sit at the edge of Half Dome looking down into Yosemite Valley [OC] [3264x2448],6v2l1x,2017-08-21 12:30:05,1503318605,[deleted],NaN,6,NaN,NaN,NaN,1,False,i.redd.it,https://i.redd.it/n9zjozff43hz.jpg,/r/EarthPorn/comments/6v2l1x/what_its_like_to_sit_at_the_edge_of_half_dome/,False,[deleted],NaN,NaN,NaN,NaN,Resolution,[3024x4032],NaN,False,NaN,NaN,NaN,False,0.0,NaN,NaN,NaN,NaN,{},{},NaN,NaN,default,False,False,False,NaN,NaN,NaN,False,False,NaN
763351,EarthPorn,t5_2sbq3,"Shot on Galaxy S7. Tučepi, Croatia",6v2ksx,2017-08-21 12:28:31,1503318511,[deleted],NaN,1,NaN,NaN,NaN,1,False,i.redd.it,https://i.redd.it/4qa5ssq843hz.jpg,/r/EarthPorn/comments/6v2ksx/shot_on_galaxy_s7_tučepi_croatia/,False,[deleted],NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,

#### Save data frame as CSV


In [8]:
filename = 'C:/Users/Chris/Desktop/DTU/4. Semester/02466 - Project Work/RedditDataWithLinks.csv'
posts_tidy_df.to_csv(filename, header=True, index=False, columns=list(posts_tidy_df.axes[1]))

## Downloading the data

#### Fetching data frame from file

In [9]:
path = 'C:/Users/Chris/Desktop/DTU/4. Semester/02466 - Project Work/RedditDataWithLinks.csv'
posts_tidy_df = pd.read_csv(path)


C:\Users\Chris\AppData\Local\Temp\ipykernel_37664\1433904771.py:2: DtypeWarning: Columns (19,21,22,26,27,28,29,30,32,33,35,41,42,43,44,47) have mixed types. Specify dtype option on import or set low_memory=False.
  posts_tidy_df = pd.read_csv(path)


In [10]:
print(posts_tidy_df.loc[[9]])

   Subreddit SubredditID  \
9  EarthPorn    t5_2sbq3   

                                               PostTitle PostID  \
9  White Sands National Monument, New Mexico [2048x1536]  swe3l   

            TimeString    PostTime    Username  ViewCount  Upvotes   Ups  \
9  2012-04-28 04:09:32  1335586172  MrMakeveli        NaN       45  50.0   

   Downs  UpvoteRatio  CommentNumber  Edited  ImageDomain  \
9    5.0          0.9              9   False  i.imgur.com   

                       ImageURL  \
9  http://i.imgur.com/qeL9g.jpg   

                                                                         Permalink  \
9  /r/EarthPorn/comments/swe3l/white_sands_national_monument_new_mexico_2048x1536/   

   IsTextPost PostText SelfTextHTML  UserCreatedTime AuthorName Distinguished  \
9       False      NaN          NaN              NaN        NaN           NaN   

  LinkFlairClass LinkFlairText AuthorBirthdate IsVideo IsMeta IsOriginal  \
9            NaN           NaN             NaN   

#### Save images from URLs

In [11]:
#Save images from data frame URL column
root_folder = "C:/Users/Chris/Desktop/DTU/4. Semester/02466 - Project Work/Images/"


def download(row):
   filename = root_folder + row['PostID'] + '.jpg'

   # create folder if it doesn't exist
   os.makedirs(os.path.dirname(filename), exist_ok = True)
   
   try:
        url = row['NewURL']

        if pd.isna(url):
            url = row['ImageURL']
            if pd.isna(url) or not url.startswith('https://i.redd.it/'):
                print(f"Skipping row {row.name} - Missing URL")
                return

            
        
        
        
        if not url.startswith('http://' ) and not url.startswith('https://' ):
            if url.startswith('////'):
                url = 'http://' + url[4:]
            else:
                url = 'http:' + url
        
        

        print(f"Downloading row {row.name} {url} to {filename}")
        r = requests.get(url, allow_redirects=True)

        if len(r.content) < 10240:  # 10kB = 10240 bytes
            print(f"Skipping row {row.name} - Image size is less than 10kB")
            return

        with open(filename, 'wb') as f:
            f.write(r.content)
        
            img = Image.open(filename)
            img = img.resize((224,224))
            img.save(filename)

   except Exception as e:
        print(f"Error occurred while processing row {row.name}: {e}")
        

posts_tidy_df.apply(download, axis=1)


Skipping row 4 - Missing URL
Skipping row 12 - Missing URL
Skipping row 13 - Missing URL
Skipping row 18 - Missing URL
Skipping row 23 - Missing URL
Skipping row 26 - Missing URL
Skipping row 27 - Missing URL
Skipping row 29 - Missing URL
Skipping row 31 - Missing URL
Skipping row 34 - Missing URL
Skipping row 39 - Missing URL
Skipping row 49 - Missing URL
Skipping row 51 - Missing URL
Skipping row 57 - Missing URL
Skipping row 59 - Missing URL
Skipping row 66 - Missing URL
Skipping row 68 - Missing URL
Skipping row 69 - Missing URL
Skipping row 72 - Missing URL
Skipping row 73 - Missing URL
Skipping row 81 - Missing URL
Skipping row 82 - Missing URL
Skipping row 84 - Missing URL
Skipping row 86 - Missing URL
Skipping row 89 - Missing URL
Skipping row 92 - Missing URL
Skipping row 96 - Missing URL
Skipping row 98 - Image size is less than 10kB
Skipping row 99 - Missing URL
Skipping row 101 - Missing URL
Skipping row 104 - Missing URL
Skipping row 106 - Image size is less than 10kB
Skip

KeyboardInterrupt: 